# LLM Model Comparison for Threat Hunting Query Generation

This notebook compares different LLM models (Llama 3.2, Mistral 7B, Gemma 2) for query generation.

## Prerequisites

Make sure you have the following models installed via Ollama:
```bash
ollama pull llama3.2
ollama pull mistral
ollama pull gemma2:9b
```

In [ ]:
import requests
import json
import time
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# Configuration
OLLAMA_URL = "http://localhost:11434/api/generate"
MODELS = [
    'llama3.2',
    'mistral',
    'gemma2:9b'
]

# Test scenarios
TEST_SCENARIOS = [
    {
        'name': 'Brute Force Attack',
        'description': 'Find failed login attempts indicating brute force attack',
        'query_type': 'spl'
    },
    {
        'name': 'Ransomware Detection',
        'description': 'Detect ransomware by finding mass file encryption events',
        'query_type': 'kql'
    },
    {
        'name': 'Data Exfiltration',
        'description': 'Identify large outbound data transfers to external destinations',
        'query_type': 'dsl'
    },
    {
        'name': 'PowerShell Obfuscation',
        'description': 'Find obfuscated PowerShell commands with base64 encoding',
        'query_type': 'spl'
    },
    {
        'name': 'Lateral Movement',
        'description': 'Detect lateral movement using RDP between hosts',
        'query_type': 'kql'
    }
]

def generate_query_with_model(model_name, description, query_type):
    """Generate query using specific model"""
    
    prompt = f"""Generate a {query_type.upper()} threat hunting query for the following:
{description}

Return ONLY the query, no explanations."""
    
    start_time = time.time()
    
    try:
        response = requests.post(
            OLLAMA_URL,
            json={
                'model': model_name,
                'prompt': prompt,
                'stream': False
            },
            timeout=60
        )
        
        generation_time = time.time() - start_time
        
        if response.status_code == 200:
            result = response.json()
            query = result.get('response', '')
            
            return {
                'success': True,
                'query': query,
                'generation_time': generation_time,
                'error': None
            }
        else:
            return {
                'success': False,
                'query': None,
                'generation_time': generation_time,
                'error': f"HTTP {response.status_code}"
            }
    except Exception as e:
        return {
            'success': False,
            'query': None,
            'generation_time': time.time() - start_time,
            'error': str(e)
        }

def evaluate_query_quality(query, query_type):
    """Basic quality evaluation of generated query"""
    if not query:
        return {'score': 0, 'has_syntax': False, 'has_fields': False, 'has_timerange': False}
    
    score = 0
    query_lower = query.lower()
    
    # Check for basic syntax
    if query_type == 'spl':
        has_syntax = 'search' in query_lower or 'index=' in query_lower
        has_fields = '|' in query or 'stats' in query_lower
        has_timerange = 'earliest' in query_lower or 'latest' in query_lower
    elif query_type == 'kql':
        has_syntax = any(table in query for table in ['SecurityEvent', 'SigninLogs', 'DeviceEvents'])
        has_fields = '|' in query or 'where' in query_lower
        has_timerange = 'ago' in query_lower or 'between' in query_lower
    else:  # dsl
        has_syntax = 'query' in query_lower or 'bool' in query_lower
        has_fields = 'must' in query_lower or 'filter' in query_lower
        has_timerange = 'range' in query_lower
    
    if has_syntax:
        score += 3
    if has_fields:
        score += 2
    if has_timerange:
        score += 2
    
    # Query length (not too short, not too long)
    length = len(query)
    if 50 < length < 500:
        score += 2
    elif length >= 500:
        score += 1
    
    return {
        'score': min(score, 10),  # Max score of 10
        'has_syntax': has_syntax,
        'has_fields': has_fields,
        'has_timerange': has_timerange,
        'length': length
    }

print("✅ Setup complete. Ready to compare models!")

## Run Model Comparison

In [ ]:
# Store results
comparison_results = []

print("🔬 Starting model comparison...\n")
print(f"Models to test: {', '.join(MODELS)}")
print(f"Scenarios: {len(TEST_SCENARIOS)}\n")

for scenario in TEST_SCENARIOS:
    print(f"\n{'='*80}")
    print(f"📝 Scenario: {scenario['name']}")
    print(f"Query Type: {scenario['query_type'].upper()}")
    print(f"{'='*80}\n")
    
    for model in MODELS:
        print(f"Testing {model}...", end=' ')
        
        result = generate_query_with_model(
            model,
            scenario['description'],
            scenario['query_type']
        )
        
        if result['success']:
            quality = evaluate_query_quality(result['query'], scenario['query_type'])
            print(f"✅ ({result['generation_time']:.2f}s, Quality: {quality['score']}/10)")
        else:
            quality = {'score': 0, 'has_syntax': False, 'has_fields': False, 'has_timerange': False, 'length': 0}
            print(f"❌ Error: {result['error']}")
        
        comparison_results.append({
            'scenario': scenario['name'],
            'query_type': scenario['query_type'],
            'model': model,
            'success': result['success'],
            'generation_time': result['generation_time'],
            'quality_score': quality['score'],
            'has_syntax': quality['has_syntax'],
            'has_fields': quality['has_fields'],
            'has_timerange': quality['has_timerange'],
            'query_length': quality.get('length', 0),
            'query': result['query']
        })

print("\n\n✅ Comparison complete!")

# Convert to DataFrame
df_results = pd.DataFrame(comparison_results)
df_results.head()

## Results Analysis

In [ ]:
# Summary statistics by model
summary = df_results.groupby('model').agg({
    'success': 'mean',
    'generation_time': 'mean',
    'quality_score': 'mean',
    'has_syntax': 'mean',
    'has_fields': 'mean',
    'has_timerange': 'mean',
    'query_length': 'mean'
}).round(2)

summary.columns = [
    'Success Rate',
    'Avg Generation Time (s)',
    'Avg Quality Score',
    'Syntax Correctness',
    'Field Usage',
    'Time Range',
    'Avg Query Length'
]

print("\n📊 MODEL COMPARISON SUMMARY")
print("="*80)
print(summary)
print("\n")

# Find best model
best_quality = summary['Avg Quality Score'].idxmax()
best_speed = summary['Avg Generation Time (s)'].idxmin()

print(f"🏆 Best Quality: {best_quality} (Score: {summary.loc[best_quality, 'Avg Quality Score']}/10)")
print(f"⚡ Fastest: {best_speed} ({summary.loc[best_speed, 'Avg Generation Time (s)']}s)")

## Visualization 1: Generation Time Comparison

In [ ]:
# Generation time comparison
fig = px.box(
    df_results,
    x='model',
    y='generation_time',
    color='model',
    title='Query Generation Time by Model',
    labels={'generation_time': 'Generation Time (seconds)', 'model': 'LLM Model'}
)
fig.update_layout(showlegend=False)
fig.show()

## Visualization 2: Quality Score Comparison

In [ ]:
# Quality score comparison
fig = go.Figure()

for model in MODELS:
    model_data = df_results[df_results['model'] == model]
    fig.add_trace(go.Bar(
        name=model,
        x=model_data['scenario'],
        y=model_data['quality_score'],
    ))

fig.update_layout(
    title='Quality Score by Scenario and Model',
    xaxis_title='Scenario',
    yaxis_title='Quality Score (0-10)',
    barmode='group'
)
fig.show()

## Visualization 3: Feature Comparison Radar Chart

In [ ]:
# Radar chart for model capabilities
categories = ['Success Rate', 'Quality Score', 'Syntax Correctness', 'Field Usage', 'Time Range']

fig = go.Figure()

for model in MODELS:
    model_summary = summary.loc[model]
    values = [
        model_summary['Success Rate'] * 10,  # Scale to 10
        model_summary['Avg Quality Score'],
        model_summary['Syntax Correctness'] * 10,
        model_summary['Field Usage'] * 10,
        model_summary['Time Range'] * 10
    ]
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories,
        fill='toself',
        name=model
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 10]
        )),
    title='Model Capabilities Comparison'
)
fig.show()

## Detailed Query Comparison (Example)

In [ ]:
# Pick a scenario and show all model outputs
scenario_name = TEST_SCENARIOS[0]['name']
scenario_results = df_results[df_results['scenario'] == scenario_name]

print(f"\n{'='*80}")
print(f"📝 Detailed Comparison for: {scenario_name}")
print(f"{'='*80}\n")

for _, row in scenario_results.iterrows():
    print(f"\n🤖 Model: {row['model']}")
    print(f"⏱️  Generation Time: {row['generation_time']:.2f}s")
    print(f"⭐ Quality Score: {row['quality_score']}/10")
    print(f"\n📜 Generated Query:")
    print("-" * 80)
    print(row['query'])
    print("-" * 80)

## Export Results

In [ ]:
# Save results to CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f'model_comparison_{timestamp}.csv'

df_results.to_csv(csv_filename, index=False)
print(f"✅ Results exported to: {csv_filename}")

# Save summary
summary_filename = f'model_summary_{timestamp}.csv'
summary.to_csv(summary_filename)
print(f"✅ Summary exported to: {summary_filename}")

## Conclusions & Recommendations

Based on the comparison results:

1. **Best Overall Model**: [To be determined from results]
2. **Fastest Model**: [To be determined from results]
3. **Most Accurate Model**: [To be determined from results]

### Recommendations:

- For **production use** where accuracy is critical: Use the model with highest quality score
- For **real-time applications**: Use the fastest model with acceptable quality
- For **complex queries**: Consider using multiple models and selecting the best output

### Next Steps:

1. Validate generated queries against real SIEM platforms
2. Collect analyst feedback on query usefulness
3. Fine-tune prompts for better quality
4. Consider ensemble approaches combining multiple models